Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [0]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0 : 3.51275
Minibatch accuracy: 6.2%
Validation accuracy: 12.8%
Minibatch loss at step 50 : 1.48703
Minibatch accuracy: 43.8%
Validation accuracy: 50.4%
Minibatch loss at step 100 : 1.04377
Minibatch accuracy: 68.8%
Validation accuracy: 67.4%
Minibatch loss at step 150 : 0.601682
Minibatch accuracy: 68.8%
Validation accuracy: 73.0%
Minibatch loss at step 200 : 0.898649
Minibatch accuracy: 75.0%
Validation accuracy: 77.8%
Minibatch loss at step 250 : 1.3637
Minibatch accuracy: 56.2%
Validation accuracy: 75.4%
Minibatch loss at step 300 : 1.41968
Minibatch accuracy: 62.5%
Validation accuracy: 76.0%
Minibatch loss at step 350 : 0.300648
Minibatch accuracy: 81.2%
Validation accuracy: 80.2%
Minibatch loss at step 400 : 1.32092
Minibatch accuracy: 56.2%
Validation accuracy: 80.4%
Minibatch loss at step 450 : 0.556701
Minibatch accuracy: 81.2%
Validation accuracy: 79.4%
Minibatch loss at step 500 : 1.65595
Minibatch accuracy: 43.8%
Validation accuracy: 79.6%

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [43]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

# Model.
def model_with_pool(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    conv = conv + layer1_biases
    max_pool1 = tf.nn.max_pool(conv, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID')
    hidden = tf.nn.relu(max_pool1)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 7 * image_size // 7 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  
  # Training computation.
  logits = model_with_pool(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model_with_pool(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model_with_pool(tf_test_dataset))

In [44]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.598219
Minibatch accuracy: 12.5%
Validation accuracy: 16.4%
Minibatch loss at step 50: 1.864993
Minibatch accuracy: 25.0%
Validation accuracy: 53.5%
Minibatch loss at step 100: 1.082548
Minibatch accuracy: 68.8%
Validation accuracy: 65.2%
Minibatch loss at step 150: 0.636230
Minibatch accuracy: 81.2%
Validation accuracy: 73.4%
Minibatch loss at step 200: 0.992086
Minibatch accuracy: 81.2%
Validation accuracy: 77.8%
Minibatch loss at step 250: 1.353827
Minibatch accuracy: 62.5%
Validation accuracy: 78.2%
Minibatch loss at step 300: 0.322821
Minibatch accuracy: 87.5%
Validation accuracy: 79.3%
Minibatch loss at step 350: 0.453936
Minibatch accuracy: 93.8%
Validation accuracy: 78.2%
Minibatch loss at step 400: 0.184385
Minibatch accuracy: 100.0%
Validation accuracy: 80.5%
Minibatch loss at step 450: 0.839338
Minibatch accuracy: 81.2%
Validation accuracy: 79.3%
Minibatch loss at step 500: 0.775262
Minibatch accuracy: 81.2%
Validation accuracy: 80.4%


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [48]:
batch_size = 256
patch_size = 5
depth1 = 20
depth2 = 50
num_hidden = 500
learn_rate=0.0001

# Model.
def model_for_high_acc(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME') #14
    conv = conv + layer1_biases
    max_pool1 = tf.nn.max_pool(conv, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID') #7
    hidden = tf.nn.relu(max_pool1)
    conv2 = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME') # 4
    conv2 = conv2 + layer2_biases
    max_pool2 = tf.nn.max_pool(conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='VALID') #2
    hidden = tf.nn.relu(max_pool2)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
    [patch_size, patch_size, num_channels, depth1], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth1]))
  layer2_weights = tf.Variable(tf.truncated_normal(
    [patch_size, patch_size, depth1, depth2], stddev=0.1))
  layer2_biases = tf.Variable(tf.zeros([depth2]))  # tf.constant(1.0, shape=[depth2]))
  layer3_weights = tf.Variable(tf.truncated_normal(
    [image_size // 14 * image_size // 14 * depth2, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
    [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Training computation.
  logits = model_for_high_acc(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(learn_rate).minimize(loss)
  optimizer = tf.train.RMSPropOptimizer(learn_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model_for_high_acc(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model_for_high_acc(tf_test_dataset))

In [49]:
num_steps = 50001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.944813
Minibatch accuracy: 11.7%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.821527
Minibatch accuracy: 5.9%
Validation accuracy: 8.3%
Minibatch loss at step 100: 2.112444
Minibatch accuracy: 29.7%
Validation accuracy: 27.7%
Minibatch loss at step 150: 1.642352
Minibatch accuracy: 59.4%
Validation accuracy: 58.3%
Minibatch loss at step 200: 1.144299
Minibatch accuracy: 73.8%
Validation accuracy: 69.7%
Minibatch loss at step 250: 0.992878
Minibatch accuracy: 72.3%
Validation accuracy: 74.7%
Minibatch loss at step 300: 0.918630
Minibatch accuracy: 75.4%
Validation accuracy: 77.6%
Minibatch loss at step 350: 0.834109
Minibatch accuracy: 76.6%
Validation accuracy: 78.9%
Minibatch loss at step 400: 0.673585
Minibatch accuracy: 82.4%
Validation accuracy: 79.4%
Minibatch loss at step 450: 0.745558
Minibatch accuracy: 77.7%
Validation accuracy: 80.8%
Minibatch loss at step 500: 0.749523
Minibatch accuracy: 78.1%
Validation accuracy: 81.1%
Min

Validation accuracy: 87.2%
Minibatch loss at step 4550: 0.499128
Minibatch accuracy: 85.9%
Validation accuracy: 87.7%
Minibatch loss at step 4600: 0.485744
Minibatch accuracy: 86.3%
Validation accuracy: 87.5%
Minibatch loss at step 4650: 0.522453
Minibatch accuracy: 84.8%
Validation accuracy: 87.6%
Minibatch loss at step 4700: 0.367025
Minibatch accuracy: 90.2%
Validation accuracy: 87.5%
Minibatch loss at step 4750: 0.307864
Minibatch accuracy: 91.8%
Validation accuracy: 87.6%
Minibatch loss at step 4800: 0.500309
Minibatch accuracy: 84.4%
Validation accuracy: 87.7%
Minibatch loss at step 4850: 0.533016
Minibatch accuracy: 84.8%
Validation accuracy: 87.5%
Minibatch loss at step 4900: 0.366097
Minibatch accuracy: 91.0%
Validation accuracy: 87.8%
Minibatch loss at step 4950: 0.459985
Minibatch accuracy: 87.1%
Validation accuracy: 87.6%
Minibatch loss at step 5000: 0.397440
Minibatch accuracy: 88.3%
Validation accuracy: 87.7%
Minibatch loss at step 5050: 0.585308
Minibatch accuracy: 84.0%

Minibatch loss at step 9050: 0.318066
Minibatch accuracy: 91.8%
Validation accuracy: 88.8%
Minibatch loss at step 9100: 0.317723
Minibatch accuracy: 89.8%
Validation accuracy: 89.0%
Minibatch loss at step 9150: 0.343224
Minibatch accuracy: 89.5%
Validation accuracy: 88.8%
Minibatch loss at step 9200: 0.394843
Minibatch accuracy: 88.7%
Validation accuracy: 88.9%
Minibatch loss at step 9250: 0.334751
Minibatch accuracy: 89.8%
Validation accuracy: 88.9%
Minibatch loss at step 9300: 0.373654
Minibatch accuracy: 89.1%
Validation accuracy: 88.9%
Minibatch loss at step 9350: 0.365539
Minibatch accuracy: 89.1%
Validation accuracy: 88.9%
Minibatch loss at step 9400: 0.305859
Minibatch accuracy: 89.8%
Validation accuracy: 88.9%
Minibatch loss at step 9450: 0.287069
Minibatch accuracy: 91.0%
Validation accuracy: 89.0%
Minibatch loss at step 9500: 0.288717
Minibatch accuracy: 92.2%
Validation accuracy: 88.8%
Minibatch loss at step 9550: 0.316324
Minibatch accuracy: 92.2%
Validation accuracy: 88.9%

Validation accuracy: 89.3%
Minibatch loss at step 13550: 0.221605
Minibatch accuracy: 92.6%
Validation accuracy: 89.6%
Minibatch loss at step 13600: 0.198273
Minibatch accuracy: 93.4%
Validation accuracy: 89.6%
Minibatch loss at step 13650: 0.321890
Minibatch accuracy: 91.4%
Validation accuracy: 89.5%
Minibatch loss at step 13700: 0.394019
Minibatch accuracy: 87.5%
Validation accuracy: 89.8%
Minibatch loss at step 13750: 0.312217
Minibatch accuracy: 91.0%
Validation accuracy: 89.5%
Minibatch loss at step 13800: 0.333241
Minibatch accuracy: 90.6%
Validation accuracy: 89.6%
Minibatch loss at step 13850: 0.346064
Minibatch accuracy: 90.2%
Validation accuracy: 89.3%
Minibatch loss at step 13900: 0.365945
Minibatch accuracy: 89.8%
Validation accuracy: 89.7%
Minibatch loss at step 13950: 0.315110
Minibatch accuracy: 90.6%
Validation accuracy: 89.5%
Minibatch loss at step 14000: 0.259342
Minibatch accuracy: 93.0%
Validation accuracy: 89.6%
Minibatch loss at step 14050: 0.295970
Minibatch accu

Minibatch loss at step 18000: 0.306129
Minibatch accuracy: 89.1%
Validation accuracy: 90.0%
Minibatch loss at step 18050: 0.395103
Minibatch accuracy: 88.3%
Validation accuracy: 89.9%
Minibatch loss at step 18100: 0.322193
Minibatch accuracy: 90.2%
Validation accuracy: 89.8%
Minibatch loss at step 18150: 0.307776
Minibatch accuracy: 91.0%
Validation accuracy: 90.2%
Minibatch loss at step 18200: 0.301619
Minibatch accuracy: 92.6%
Validation accuracy: 90.1%
Minibatch loss at step 18250: 0.303578
Minibatch accuracy: 91.0%
Validation accuracy: 90.1%
Minibatch loss at step 18300: 0.244039
Minibatch accuracy: 92.2%
Validation accuracy: 90.1%
Minibatch loss at step 18350: 0.357627
Minibatch accuracy: 87.9%
Validation accuracy: 90.1%
Minibatch loss at step 18400: 0.273759
Minibatch accuracy: 91.4%
Validation accuracy: 90.1%
Minibatch loss at step 18450: 0.277325
Minibatch accuracy: 92.6%
Validation accuracy: 90.2%
Minibatch loss at step 18500: 0.181947
Minibatch accuracy: 95.7%
Validation accu

...

Validation accuracy: 90.7%
Minibatch loss at step 31450: 0.273683
Minibatch accuracy: 92.2%
Validation accuracy: 90.6%
Minibatch loss at step 31500: 0.233106
Minibatch accuracy: 93.4%
Validation accuracy: 90.7%
Minibatch loss at step 31550: 0.246492
Minibatch accuracy: 93.4%
Validation accuracy: 90.6%
Minibatch loss at step 31600: 0.284502
Minibatch accuracy: 91.0%
Validation accuracy: 90.7%
Minibatch loss at step 31650: 0.205140
Minibatch accuracy: 92.6%
Validation accuracy: 90.4%
Minibatch loss at step 31700: 0.271310
Minibatch accuracy: 93.4%
Validation accuracy: 90.7%
Minibatch loss at step 31750: 0.224160
Minibatch accuracy: 93.0%
Validation accuracy: 90.7%
Minibatch loss at step 31800: 0.268820
Minibatch accuracy: 91.4%
Validation accuracy: 90.8%
Minibatch loss at step 31850: 0.241727
Minibatch accuracy: 92.2%
Validation accuracy: 90.7%
Minibatch loss at step 31900: 0.289418
Minibatch accuracy: 91.8%
Validation accuracy: 90.6%
Minibatch loss at step 31950: 0.245839
Minibatch accu

Minibatch loss at step 35900: 0.216955
Minibatch accuracy: 93.8%
Validation accuracy: 90.8%
Minibatch loss at step 35950: 0.203609
Minibatch accuracy: 93.8%
Validation accuracy: 90.8%
Minibatch loss at step 36000: 0.214305
Minibatch accuracy: 93.4%
Validation accuracy: 90.8%
Minibatch loss at step 36050: 0.237988
Minibatch accuracy: 94.1%
Validation accuracy: 90.8%
Minibatch loss at step 36100: 0.193949
Minibatch accuracy: 94.5%
Validation accuracy: 90.8%
Minibatch loss at step 36150: 0.362284
Minibatch accuracy: 87.5%
Validation accuracy: 90.7%
Minibatch loss at step 36200: 0.251589
Minibatch accuracy: 93.0%
Validation accuracy: 90.8%
Minibatch loss at step 36250: 0.174678
Minibatch accuracy: 94.1%
Validation accuracy: 90.7%
Minibatch loss at step 36300: 0.278886
Minibatch accuracy: 91.4%
Validation accuracy: 90.7%
Minibatch loss at step 36350: 0.188942
Minibatch accuracy: 94.1%
Validation accuracy: 90.6%
Minibatch loss at step 36400: 0.304444
Minibatch accuracy: 89.5%
Validation accu

Validation accuracy: 90.7%
Minibatch loss at step 40400: 0.294879
Minibatch accuracy: 93.4%
Validation accuracy: 90.7%
Minibatch loss at step 40450: 0.283827
Minibatch accuracy: 92.6%
Validation accuracy: 90.9%
Minibatch loss at step 40500: 0.269584
Minibatch accuracy: 94.1%
Validation accuracy: 90.7%
Minibatch loss at step 40550: 0.165756
Minibatch accuracy: 94.1%
Validation accuracy: 90.9%
Minibatch loss at step 40600: 0.294324
Minibatch accuracy: 91.4%
Validation accuracy: 90.8%
Minibatch loss at step 40650: 0.189471
Minibatch accuracy: 94.5%
Validation accuracy: 90.8%
Minibatch loss at step 40700: 0.187469
Minibatch accuracy: 93.4%
Validation accuracy: 91.0%
Minibatch loss at step 40750: 0.184887
Minibatch accuracy: 93.8%
Validation accuracy: 90.9%
Minibatch loss at step 40800: 0.282501
Minibatch accuracy: 92.6%
Validation accuracy: 90.8%
Minibatch loss at step 40850: 0.210074
Minibatch accuracy: 93.8%
Validation accuracy: 91.0%
Minibatch loss at step 40900: 0.212830
Minibatch accu

Minibatch loss at step 44850: 0.148506
Minibatch accuracy: 95.3%
Validation accuracy: 90.9%
Minibatch loss at step 44900: 0.229843
Minibatch accuracy: 93.4%
Validation accuracy: 90.8%
Minibatch loss at step 44950: 0.266160
Minibatch accuracy: 93.4%
Validation accuracy: 90.8%
Minibatch loss at step 45000: 0.233795
Minibatch accuracy: 91.4%
Validation accuracy: 90.9%
Minibatch loss at step 45050: 0.219356
Minibatch accuracy: 92.6%
Validation accuracy: 90.9%
Minibatch loss at step 45100: 0.200071
Minibatch accuracy: 93.8%
Validation accuracy: 90.8%
Minibatch loss at step 45150: 0.171382
Minibatch accuracy: 95.3%
Validation accuracy: 90.8%
Minibatch loss at step 45200: 0.182413
Minibatch accuracy: 94.1%
Validation accuracy: 91.1%
Minibatch loss at step 45250: 0.193989
Minibatch accuracy: 93.8%
Validation accuracy: 90.9%
Minibatch loss at step 45300: 0.172928
Minibatch accuracy: 93.0%
Validation accuracy: 91.1%
Minibatch loss at step 45350: 0.210443
Minibatch accuracy: 93.0%
Validation accu

Validation accuracy: 91.1%
Minibatch loss at step 49350: 0.154607
Minibatch accuracy: 94.9%
Validation accuracy: 91.0%
Minibatch loss at step 49400: 0.207015
Minibatch accuracy: 93.0%
Validation accuracy: 90.9%
Minibatch loss at step 49450: 0.261042
Minibatch accuracy: 90.2%
Validation accuracy: 90.7%
Minibatch loss at step 49500: 0.254130
Minibatch accuracy: 93.0%
Validation accuracy: 90.8%
Minibatch loss at step 49550: 0.189570
Minibatch accuracy: 93.0%
Validation accuracy: 91.0%
Minibatch loss at step 49600: 0.128227
Minibatch accuracy: 96.9%
Validation accuracy: 90.8%
Minibatch loss at step 49650: 0.231273
Minibatch accuracy: 93.8%
Validation accuracy: 90.8%
Minibatch loss at step 49700: 0.238832
Minibatch accuracy: 91.8%
Validation accuracy: 90.8%
Minibatch loss at step 49750: 0.195048
Minibatch accuracy: 93.8%
Validation accuracy: 91.0%
Minibatch loss at step 49800: 0.177625
Minibatch accuracy: 94.9%
Validation accuracy: 90.8%
Minibatch loss at step 49850: 0.211784
Minibatch accu